In [ ]:
# Retrieving GEOData
# Download the code from www.py4e.com/code3/opengeo.zip - then unzip the file and edit where.data to add 
# an address nearby where you live - don't reveal where you live. Then run the geoload.py to lookup all 
# of the entries in where.data (including the new one) and produce the geodata.sqlite. Then run geodump.py 
# to read the database and produce where.js. You can run the programs and then scroll back to take your 
# screen shots when the program finishes. Then open where.html to visualize the map. 

In [25]:
# geoload.py

import urllib.request, urllib.parse, urllib.error
import http
import sqlite3
import json
import time
import ssl
import sys

# https://py4e-data.dr-chuck.net/opengeo?q=Ann+Arbor%2C+MI
serviceurl = 'https://py4e-data.dr-chuck.net/opengeo?'

# Additional detail for urllib
# http.client.HTTPConnection.debuglevel = 1

conn = sqlite3.connect('opengeo.sqlite')
cur = conn.cursor()

# Drop and create the table 
cur.executescript('''
            DROP TABLE IF EXISTS Locations;

            CREATE TABLE Locations (
                id     INTEGER PRIMARY KEY,
                address   TEXT,
                lat    TEXT, 
                lng    TEXT,
                geodata TEXT
                )
                ''')

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

fh = open("where.data")
count = 0
nofound = 0
for line in fh:
    if count > 100 :
        print('Retrieved 100 locations, restart to retrieve more')
        break

    address = line.strip()
    print('')
    cur.execute("SELECT geodata FROM Locations WHERE address= ?",
        (memoryview(address.encode()), ))

    try:
        data = cur.fetchone()[0]
        print("Found in database", address)
        continue
    except:
        pass

    parms = dict()
    parms['q'] = address

    url = serviceurl + urllib.parse.urlencode(parms)

    print('Retrieving', url)
    uh = urllib.request.urlopen(url, context=ctx)
    data = uh.read().decode()
    print('Retrieved', len(data), 'characters', data[:20].replace('\n', ' '))
    count = count + 1

    try:
        js = json.loads(data)
    except:
        print(data)  # We print in case unicode causes an error
        continue

    if not js or 'features' not in js:
        print('==== Download error ===')
        print(data)
        break

    if len(js['features']) == 0:
        print('==== Object not found ====')
        nofound = nofound + 1

    cur.execute('''INSERT INTO Locations (address, geodata)
        VALUES ( ?, ? )''',
        (memoryview(address.encode()), memoryview(data.encode()) ) )

    conn.commit()

    if count % 10 == 0 :
        print('Pausing for a bit...')
        time.sleep(1)

if nofound > 0:
    print('Number of features for which the location could not be found:', nofound)

print("Run geodump.py to read the data from the database so you can vizualize it on a map.")




Retrieving https://py4e-data.dr-chuck.net/opengeo?q=AGH+University+of+Science+and+Technology
Retrieved 1533 characters {"type":"FeatureColl

Retrieving https://py4e-data.dr-chuck.net/opengeo?q=Nelson+Mandela+University
Retrieved 1607 characters {"type":"FeatureColl

Retrieving https://py4e-data.dr-chuck.net/opengeo?q=Academy+of+Fine+Arts+Warsaw+Poland
Retrieved 1682 characters {"type":"FeatureColl

Retrieving https://py4e-data.dr-chuck.net/opengeo?q=American+University+in+Cairo
Retrieved 1545 characters {"type":"FeatureColl

Retrieving https://py4e-data.dr-chuck.net/opengeo?q=Arizona+State+University
Retrieved 1534 characters {"type":"FeatureColl

Retrieving https://py4e-data.dr-chuck.net/opengeo?q=Athens+Information+Technology
Retrieved 1424 characters {"type":"FeatureColl

Retrieving https://py4e-data.dr-chuck.net/opengeo?q=BITS+Pilani
Retrieved 1285 characters {"type":"FeatureColl

Retrieving https://py4e-data.dr-chuck.net/opengeo?q=Babcock+University
Retrieved 1376 characters {"ty

In [31]:
# geodump.py

import sqlite3
import json
import codecs

conn = sqlite3.connect('opengeo.sqlite')
cur = conn.cursor()

cur.execute('SELECT * FROM Locations')
fhand = codecs.open('where.js', 'w', "utf-8")
print()
fhand.write("myData = [\n")
count = 0
for row in cur :
    data = str(row[1])
    try: js = json.loads(str(data))
    except: continue

    if len(js['features']) == 0: continue

    try:
        lat = js['features'][0]['geometry']['coordinates'][1]
        lng = js['features'][0]['geometry']['coordinates'][0]
        where = js['features'][0]['properties']['display_name']
        where = where.replace("'", "")
    except:
        print('Unexpected format')
        print(js)

    try :
        print(where, lat, lng)

        count = count + 1
        if count > 1 : fhand.write(",\n")
        output = "["+str(lat)+","+str(lng)+", '"+where+"']"
        fhand.write(output)
    except:
        continue
        
# Nelson Mandela University location
if count > 0:
    fhand.write(",\n")
output = "[-33.9608, 25.6022, 'Nelson Mandela University, South Africa']"
fhand.write(output)
print(output)
fhand.write("\n];\n")
cur.close()
fhand.close()
print(count + 1, "records written to where.js")
print("Open where.html to view the data in a browser")




[-33.9608, 25.6022, 'Nelson Mandela University, South Africa']
1 records written to where.js
Open where.html to view the data in a browser
